In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [2]:
te= pd.read_csv('input/te/te_loc_cleaned.csv')
te.head()


,games,year,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,def_rk,h/a_1,surface_0,points
0,1,2014,AD Denham,HOU,14,JAX,0,0,0,0,0.0,0.0,30,0.0,1.0,0.0
1,2,2014,AD Denham,HOU,15,IND,0,0,0,0,0.0,0.0,5,0.0,0.0,0.0
2,3,2014,AD Denham,HOU,16,BAL,0,0,0,0,0.0,0.0,4,1.0,0.0,0.0
3,4,2014,AD Denham,HOU,17,JAX,0,0,0,0,0.0,0.0,30,1.0,0.0,0.0
4,1,2016,AJ Derby,NE,1,ARI,0,0,0,0,0.0,0.0,1,0.0,1.0,0.0


In [3]:
te['year'].unique()

array([2014, 2016, 2017, 2018, 2012, 2013, 2015], dtype=int64)

In [4]:
te_18=te[te['year']==2018]
te_18['Week'].unique()

array([2, 3, 6, 1, 4, 5, 7, 8], dtype=int64)

In [5]:
te.sort_values(['Name','year','games'], inplace=True)
te.head()

,games,year,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,def_rk,h/a_1,surface_0,points
0,1,2014,AD Denham,HOU,14,JAX,0,0,0,0,0.0,0.0,30,0.0,1.0,0.0
1,2,2014,AD Denham,HOU,15,IND,0,0,0,0,0.0,0.0,5,0.0,0.0,0.0
2,3,2014,AD Denham,HOU,16,BAL,0,0,0,0,0.0,0.0,4,1.0,0.0,0.0
3,4,2014,AD Denham,HOU,17,JAX,0,0,0,0,0.0,0.0,30,1.0,0.0,0.0
4,1,2016,AJ Derby,NE,1,ARI,0,0,0,0,0.0,0.0,1,0.0,1.0,0.0


In [6]:
te.reset_index(drop=True, inplace=True)
te.tail()

,games,year,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,def_rk,h/a_1,surface_0,points
9233,26,2014,Zach Sudfeld,NYJ,13,MIA,1,1,20,0,20.0,20.0,24,1.0,0.0,2.0
9234,27,2014,Zach Sudfeld,NYJ,14,MIN,0,0,0,0,0.0,0.0,29,1.0,0.0,0.0
9235,28,2014,Zach Sudfeld,NYJ,15,TEN,0,0,0,0,0.0,0.0,22,0.0,1.0,0.0
9236,29,2014,Zach Sudfeld,NYJ,16,NE,1,1,6,0,6.0,6.0,27,1.0,0.0,0.6
9237,30,2014,Zach Sudfeld,NYJ,17,MIA,2,2,44,0,22.0,22.0,24,0.0,1.0,4.4


In [7]:
te.shape

(9238, 16)

In [10]:
def create_sorted_dict (df):
    df_sorted = df.sort_values(['Name','games'])
    sorted_dict = {}
    for player in df_sorted['Name']:
        sorted_dict['{}'.format(player)] = {'year':df_sorted[df_sorted['Name']==player]['year'],
                                            'Opponent':df_sorted[df_sorted['Name']==player]['Opponent'],
                                            'points':df_sorted[df_sorted['Name']==player]['points'],
                                            'def_rk':df_sorted[df_sorted['Name']==player]['def_rk'],
                                            'surface_0':df_sorted[df_sorted['Name']==player]['surface_0'],
                                            'h/a_1':df_sorted[df_sorted['Name']== player]['h/a_1'],
                                            'games': df_sorted[df_sorted['Name']==player]['games']
                                           }
       
    return sorted_dict

In [11]:
rb_sorted_dict = create_sorted_dict(rb)

In [140]:
# Create a data frame to investigate the points attribute of the dataset. 


def get_points_df(df):
    total_points = df['points'].sum()
    total_games = df['points'].count()

    val_lst = []
    sal_lst = []
    games_lst = []
    wght_avg=[]
    players = []
    avg_lst=[]

    for name in list(df['Name'].unique()):
        players.append(name)
        player_games = df[df['Name'] ==name].describe().loc['count'][0]
        games_lst.append(player_games)
        avg_points = round(df[df['Name'] ==name].describe()['points'].mean(),2)
        avg_lst.append(avg_points)
        wght_avg.append((round(avg_points + (avg_points* (player_games/total_games)),2)))
        sal_avg= round(df[df['Name'] ==name].describe()['salary'].mean(),2)
        sal_lst.append(sal_avg)
        val = round(sal_avg/avg_points,2)
        val_lst.append(val)
     
    df_eng = pd.DataFrame()
    df_eng['player'] = (df['Name'].unique())
    df_eng['games'] = games_lst
    df_eng['avg_points']= avg_lst
    df_eng['wght_avg'] = wght_avg
    df_eng['sal_avg'] = sal_lst
    df_eng['val'] = val_lst
    df_eng.head()
    
    return df_eng

## Group Dataframes

In [7]:
# A function to create a dictonary of grouped data frames

def grouped_dict(csv):
    
    df= pd.read_csv(csv)
    

    yearly_dict = {}
    for year in list(df['year'].unique()):
        yearly_dict['player_{}'.format(year)] = (df[df['year']==year]).drop('year',axis=1).groupby(['Name','games','Opponent']).sum()
        


    return yearly_dict

        
        

In [8]:
def conv_atts(grouped_dict):
    for group in grouped_dict:
        grouped_dict['{}'.format(group)]['h/a_1'] = grouped_dict['{}'.format(group)]['h/a_1'].astype('category')
        grouped_dict['{}'.format(group)]['surface_0'] = grouped_dict['{}'.format(group)]['surface_0'].astype('category')
    
    return grouped_dict

In [9]:
te_group = grouped_dict('input/te/te_loc_cleaned.csv')

In [10]:
te_group = conv_atts(te_group)


In [11]:
te_group['player_2013'].head()

Week  tar  rec  yds  td  yds/tar  yds/rec  \
Name            games Opponent                                              
Adrien Robinson 3     DET         16    0    0    0   0      0.0      0.0   
Alex Smith      13    CHI          1    0    0    0   0      0.0      0.0   
                14    PIT          2    1    0    0   0      0.0      0.0   
                15    GB           3    0    0    0   0      0.0      0.0   
                16    CLE          4    0    0    0   0      0.0      0.0   

                                def_rk h/a_1 surface_0  points  
Name            games Opponent                                  
Adrien Robinson 3     DET           15   0.0       0.0     0.0  
Alex Smith      13    CHI           20   0.0       1.0     0.0  
                14    PIT            9   1.0       0.0     0.0  
                15    GB             2   1.0       0.0     0.0  
                16    CLE            8   0.0       1.0     0.0

In [12]:
def drop_col (grouped_df):
    for group in grouped_df:
        grouped_df[group].drop('h/a_1', axis=1, inplace=True)
        grouped_df[group].drop('surface_0',axis=1, inplace=True)
        grouped_df[group].drop('points',axis=1, inplace=True)
        grouped_df[group].drop('def_rk',axis=1, inplace=True)
        grouped_df[group].drop('Week', axis=1, inplace=True)

    return grouped_df

In [13]:
te_group= drop_col(te_group)

In [14]:
te_group['player_2015'].columns

Index(['tar', 'rec', 'yds', 'td', 'yds/tar', 'yds/rec'], dtype='object')

## Current Stats
Weekly moving average for every year from 2015 through 2018

In [15]:
# Week 1 stats are player's career average through 2014 season
# Populate current dataframe with current year averages of players

def insert_data (grouped_df):
    new_df = pd.DataFrame(index=grouped_df.index)
    for col in grouped_df: # Iterate through every attribute in the team_stats dataframe
        if col not in list(['def_rk','surface_0','h/a_1','games','points']):
            points=[]  # Create an empty list to hold the season average of the current attribute
            for player in grouped_df.index.levels[0]:  # Iterate through every player in the team_stats df
                total =[]# create an empty list to hold the weekly attributes valu3

                for week in grouped_df.loc[player].index: # Iterate throough every week for the current player
                    total.append(grouped_df.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
                    points.append(round(np.mean(total),2))  # Add the average of the total list to the season average list

            new_df['{}_avg'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe
        else:
            None
    return new_df

In [16]:
weekly_2012 = insert_data(te_group['player_2012'])
weekly_2013 = insert_data(te_group['player_2013'])
weekly_2014 = insert_data(te_group['player_2014'])
weekly_2015 = insert_data(te_group['player_2015'])
weekly_2016 = insert_data(te_group['player_2016'])
weekly_2017 = insert_data(te_group['player_2017'])
weekly_2018 = insert_data(te_group['player_2018'])

In [20]:
weekly_2012

tar_avg  rec_avg  yds_avg  td_avg  \
Name            games Opponent                                      
Aaron Hernandez 1     TEN          7.00     6.00    59.00    1.00   
                2     ARI          4.00     3.00    29.50    0.50   
                3     SEA          5.67     4.00    29.67    0.67   
                4     NYJ          6.00     4.25    35.75    0.50   
                5     NYJ          5.40     3.80    35.80    0.40   

                                yds/tar_avg  yds/rec_avg  
Name            games Opponent                            
Aaron Hernandez 1     TEN              8.40         9.80  
                2     ARI              4.20         4.90  
                3     SEA              3.90         4.93  
                4     NYJ              4.85         6.40  
                5     NYJ              6.28         8.72

In [17]:
def shift_dict(df):

    shift_dict = {}
    df.reset_index(inplace=True)
    for player in df['Name'].unique():
        shift_dict[player]=df[df['Name']==player].shift(1).fillna(0)
    df_shifted = pd.concat(shift_dict)
    df_shifted.drop('Name', axis=1, inplace=True)
    
    
    
    return df_shifted

In [18]:
player_2012_shifted = shift_dict(weekly_2012)
player_2013_shifted = shift_dict(weekly_2013)
player_2014_shifted = shift_dict(weekly_2014)
player_2015_shifted = shift_dict(weekly_2015)
player_2016_shifted = shift_dict(weekly_2016)
player_2017_shifted = shift_dict(weekly_2017)
player_2018_shifted = shift_dict(weekly_2018)

In [19]:
player_2012_shifted['year']=2012
player_2013_shifted['year']=2013
player_2014_shifted['year']=2014
player_2015_shifted['year']=2015
player_2016_shifted['year']=2016
player_2017_shifted['year']=2017
player_2018_shifted['year']=2018

In [24]:
player_2018_shifted.head()

games Opponent  tar_avg  rec_avg  yds_avg  td_avg  \
AJ Derby        0    0.0        0      0.0      0.0      0.0     0.0   
                1   22.0      NYJ      2.0      1.0     19.0     1.0   
Anthony Firkser 2    0.0        0      0.0      0.0      0.0     0.0   
Antonio Gates   3    0.0        0      0.0      0.0      0.0     0.0   
                4   89.0       KC      3.0      2.0     16.0     0.0   

                   yds/tar_avg  yds/rec_avg  year  
AJ Derby        0          0.0          0.0  2018  
                1          9.5         19.0  2018  
Anthony Firkser 2          0.0          0.0  2018  
Antonio Gates   3          0.0          0.0  2018  
                4          5.3          8.0  2018

In [20]:
player_weekly_avg = pd.concat([player_2012_shifted,player_2013_shifted, player_2014_shifted,player_2015_shifted, player_2016_shifted, player_2017_shifted, player_2018_shifted])

player_weekly_avg.head()

games Opponent  tar_avg  rec_avg  yds_avg  td_avg  \
Aaron Hernandez 0    0.0        0     0.00     0.00     0.00    0.00   
                1    1.0      TEN     7.00     6.00    59.00    1.00   
                2    2.0      ARI     4.00     3.00    29.50    0.50   
                3    3.0      SEA     5.67     4.00    29.67    0.67   
                4    4.0      NYJ     6.00     4.25    35.75    0.50   

                   yds/tar_avg  yds/rec_avg  year  
Aaron Hernandez 0         0.00         0.00  2012  
                1         8.40         9.80  2012  
                2         4.20         4.90  2012  
                3         3.90         4.93  2012  
                4         4.85         6.40  2012

In [21]:
player_weekly_avg.reset_index(inplace=True)
player_weekly_avg.head()

,level_0,level_1,games,Opponent,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,year
0,Aaron Hernandez,0,0.0,0,0.00,0.00,0.00,0.00,0.00,0.00,2012
1,Aaron Hernandez,1,1.0,TEN,7.00,6.00,59.00,1.00,8.40,9.80,2012
2,Aaron Hernandez,2,2.0,ARI,4.00,3.00,29.50,0.50,4.20,4.90,2012
3,Aaron Hernandez,3,3.0,SEA,5.67,4.00,29.67,0.67,3.90,4.93,2012
4,Aaron Hernandez,4,4.0,NYJ,6.00,4.25,35.75,0.50,4.85,6.40,2012


In [22]:
player_weekly_avg = player_weekly_avg.rename(columns={'level_0':'Name'}).drop('level_1', axis=1)
player_weekly_avg.head()

,Name,games,Opponent,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,year
0,Aaron Hernandez,0.0,0,0.00,0.00,0.00,0.00,0.00,0.00,2012
1,Aaron Hernandez,1.0,TEN,7.00,6.00,59.00,1.00,8.40,9.80,2012
2,Aaron Hernandez,2.0,ARI,4.00,3.00,29.50,0.50,4.20,4.90,2012
3,Aaron Hernandez,3.0,SEA,5.67,4.00,29.67,0.67,3.90,4.93,2012
4,Aaron Hernandez,4.0,NYJ,6.00,4.25,35.75,0.50,4.85,6.40,2012


In [23]:
player_weekly_avg.sort_values(['Name','year','games'],inplace=True)
player_weekly_avg.head()

,Name,games,Opponent,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,year
2946,AD Denham,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,2014
2947,AD Denham,1.0,JAX,0.0,0.0,0.0,0.0,0.0,0.0,2014
2948,AD Denham,2.0,IND,0.0,0.0,0.0,0.0,0.0,0.0,2014
2949,AD Denham,3.0,BAL,0.0,0.0,0.0,0.0,0.0,0.0,2014
5874,AJ Derby,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,2016


In [24]:
player_weekly_avg.drop(['Name','games','Opponent'], axis=1, inplace=True)

In [25]:
player_weekly_avg.head()

,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,year
2946,0.0,0.0,0.0,0.0,0.0,0.0,2014
2947,0.0,0.0,0.0,0.0,0.0,0.0,2014
2948,0.0,0.0,0.0,0.0,0.0,0.0,2014
2949,0.0,0.0,0.0,0.0,0.0,0.0,2014
5874,0.0,0.0,0.0,0.0,0.0,0.0,2016


In [26]:
player_weekly_avg.reset_index(inplace=True, drop=True)
player_weekly_avg.head()

,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,year
0,0.0,0.0,0.0,0.0,0.0,0.0,2014
1,0.0,0.0,0.0,0.0,0.0,0.0,2014
2,0.0,0.0,0.0,0.0,0.0,0.0,2014
3,0.0,0.0,0.0,0.0,0.0,0.0,2014
4,0.0,0.0,0.0,0.0,0.0,0.0,2016


## Recent Player Stats
Calculate the 4 game moving average of all attributes for each player and insert resluts into positional dataframe

In [27]:
# A function designed to calcuate a player's 4 game moving average in seasons 2015,2016,2017,2018

def moving_average(grouped_df, games):
    new_grouped_df = pd.DataFrame(index = grouped_df.index)
    for col in grouped_df:  # Iterate through every attribute in the team_stats dataframe
        if col not in list(['def_rk','surface_0','h/a_1','games','points']):
            ma4 = []  # Create an empty list to hold the season average of the current attribute
            for player in grouped_df.index.levels[0]:  # Iterate through every player in the team_stats df
                total = [] # INstantiate a deque container with a max length of 3 to hold the three most current games for player
                count = 0 # Counter to track week number 
                for week in grouped_df.loc[player].index: # Iterate through every week for relevant player d
                    if count < games: # check count value to start 3 game moving average
                        ma4.append(0)  # week 1 through 3 have 3 game movong average of 0
                        total.append(grouped_df.loc[player].loc[week][col])  # Add the value current player's attribute total from the left
                        count += 1  # Increase count
                    else:  # Once 3 game moving average is avaliable
                        ma4.append(np.mean(total))  # Add the average of the 3 games held in totals 3 game moivng average list
                        total.append(grouped_df.loc[player].loc[week][col]) # Replace 3 set block with a first in first out process
        else:
            None
        new_grouped_df['{}_ma'.format(col.lower())] = ma4  # Populate databse with 3 game moving average list
       

    return new_grouped_df   

In [28]:
player_ma_2012 = moving_average(te_group['player_2012'],4)
player_ma_2013 = moving_average(te_group['player_2013'],4)
player_ma_2014 = moving_average(te_group['player_2014'],4)
player_ma_2015 = moving_average(te_group['player_2015'],4)
player_ma_2016 = moving_average(te_group['player_2016'],4)
player_ma_2017 = moving_average(te_group['player_2017'],4)
player_ma_2018 = moving_average(te_group['player_2018'],4)

In [29]:
yearly_ma_grp = pd.concat([player_ma_2012, player_ma_2013])
yearly_ma_grp = pd.concat([yearly_ma_grp, player_ma_2014])
yearly_ma_grp = pd.concat([yearly_ma_grp, player_ma_2015])
yearly_ma_grp = pd.concat([yearly_ma_grp, player_ma_2016])
yearly_ma_grp = pd.concat([yearly_ma_grp,player_ma_2017])
yearly_ma_grp = pd.concat([yearly_ma_grp,player_ma_2018])
yearly_ma_grp.head()

tar_ma  rec_ma  yds_ma  td_ma  yds/tar_ma  \
Name            games Opponent                                              
Aaron Hernandez 1     TEN          0.0    0.00    0.00    0.0        0.00   
                2     ARI          0.0    0.00    0.00    0.0        0.00   
                3     SEA          0.0    0.00    0.00    0.0        0.00   
                4     NYJ          0.0    0.00    0.00    0.0        0.00   
                5     NYJ          6.0    4.25   35.75    0.5        4.85   

                                yds/rec_ma  
Name            games Opponent              
Aaron Hernandez 1     TEN              0.0  
                2     ARI              0.0  
                3     SEA              0.0  
                4     NYJ              0.0  
                5     NYJ              6.4

In [30]:
yearly_ma_grp.reset_index(inplace=True)
yearly_ma_grp.head()

,Name,games,Opponent,tar_ma,rec_ma,yds_ma,td_ma,yds/tar_ma,yds/rec_ma
0,Aaron Hernandez,1,TEN,0.0,0.00,0.00,0.0,0.00,0.0
1,Aaron Hernandez,2,ARI,0.0,0.00,0.00,0.0,0.00,0.0
2,Aaron Hernandez,3,SEA,0.0,0.00,0.00,0.0,0.00,0.0
3,Aaron Hernandez,4,NYJ,0.0,0.00,0.00,0.0,0.00,0.0
4,Aaron Hernandez,5,NYJ,6.0,4.25,35.75,0.5,4.85,6.4


In [31]:
yearly_ma_grp.sort_values(['Name','games'], inplace=True)
yearly_ma_grp.head()

,Name,games,Opponent,tar_ma,rec_ma,yds_ma,td_ma,yds/tar_ma,yds/rec_ma
2946,AD Denham,1,JAX,0.0,0.0,0.0,0.0,0.0,0.0
2947,AD Denham,2,IND,0.0,0.0,0.0,0.0,0.0,0.0
2948,AD Denham,3,BAL,0.0,0.0,0.0,0.0,0.0,0.0
2949,AD Denham,4,JAX,0.0,0.0,0.0,0.0,0.0,0.0
5874,AJ Derby,1,ARI,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
yearly_ma_grp.reset_index(drop=True, inplace=True)
yearly_ma_grp.head()

,Name,games,Opponent,tar_ma,rec_ma,yds_ma,td_ma,yds/tar_ma,yds/rec_ma
0,AD Denham,1,JAX,0.0,0.0,0.0,0.0,0.0,0.0
1,AD Denham,2,IND,0.0,0.0,0.0,0.0,0.0,0.0
2,AD Denham,3,BAL,0.0,0.0,0.0,0.0,0.0,0.0
3,AD Denham,4,JAX,0.0,0.0,0.0,0.0,0.0,0.0
4,AJ Derby,1,ARI,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
player_weekly_avg.shape, yearly_ma_grp.shape

((9238, 7), (9238, 9))

In [33]:
yearly_ma_grp[yearly_ma_grp['Name']=='Rob Gronkowski']

,Name,games,Opponent,tar_ma,rec_ma,yds_ma,td_ma,yds/tar_ma,yds/rec_ma
7621,Rob Gronkowski,1,TEN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7622,Rob Gronkowski,2,ARI,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7623,Rob Gronkowski,3,BAL,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7624,Rob Gronkowski,4,BUF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7625,Rob Gronkowski,5,DEN,7.250000,4.750000,65.000000,0.750000,8.700000,13.450000
7626,Rob Gronkowski,6,SEA,6.800000,4.600000,59.000000,0.600000,8.360000,12.520000
7627,Rob Gronkowski,7,NYJ,7.166667,4.833333,59.333333,0.500000,8.100000,12.133333
7628,Rob Gronkowski,8,LAR,7.428571,5.000000,62.000000,0.714286,8.185714,12.257143
7629,Rob Gronkowski,9,BUF,8.125000,5.375000,72.500000,0.875000,8.562500,13.000000
7630,Rob Gronkowski,10,IND,7.666667,5.111111,67.888889,0.888889,8.477778,12.700000


In [39]:
temp= pd.concat([player_weekly_avg,yearly_ma_grp],axis=1)
temp.isnull().sum()

tar_avg        0
rec_avg        0
yds_avg        0
td_avg         0
yds/tar_avg    0
yds/rec_avg    0
year           0
Name           0
games          0
Opponent       0
tar_ma         0
rec_ma         0
yds_ma         0
td_ma          0
yds/tar_ma     0
yds/rec_ma     0
dtype: int64

In [40]:
temp.head()

,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,year,Name,games,Opponent,tar_ma,rec_ma,yds_ma,td_ma,yds/tar_ma,yds/rec_ma
0,0.0,0.0,0.0,0.0,0.0,0.0,2014,AD Denham,1,JAX,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,2014,AD Denham,2,IND,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,2014,AD Denham,3,BAL,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,2014,AD Denham,4,JAX,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,2016,AJ Derby,1,ARI,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
temp.isnull().sum()

tar_avg        0
rec_avg        0
yds_avg        0
td_avg         0
yds/tar_avg    0
yds/rec_avg    0
year           0
Name           0
games          0
Opponent       0
tar_ma         0
rec_ma         0
yds_ma         0
td_ma          0
yds/tar_ma     0
yds/rec_ma     0
dtype: int64

In [42]:
cols = temp.columns.tolist()
cols = cols[6:10] + cols[:6] + cols[10:]
temp = temp[cols]
temp.head()

,year,Name,games,Opponent,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,tar_ma,rec_ma,yds_ma,td_ma,yds/tar_ma,yds/rec_ma
0,2014,AD Denham,1,JAX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,AD Denham,2,IND,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,AD Denham,3,BAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,AD Denham,4,JAX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,AJ Derby,1,ARI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
temp.tail()

,year,Name,games,Opponent,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,tar_ma,rec_ma,yds_ma,td_ma,yds/tar_ma,yds/rec_ma
9233,2014,Zach Sudfeld,26,MIA,0.27,0.09,1.36,0.0,1.36,1.36,0.272727,0.090909,1.363636,0.0,1.363636,1.363636
9234,2014,Zach Sudfeld,27,MIN,0.33,0.17,2.92,0.0,2.92,2.92,0.333333,0.166667,2.916667,0.0,2.916667,2.916667
9235,2014,Zach Sudfeld,28,TEN,0.31,0.15,2.69,0.0,2.69,2.69,0.307692,0.153846,2.692308,0.0,2.692308,2.692308
9236,2014,Zach Sudfeld,29,NE,0.29,0.14,2.50,0.0,2.50,2.50,0.285714,0.142857,2.500000,0.0,2.500000,2.500000
9237,2014,Zach Sudfeld,30,MIA,0.33,0.20,2.73,0.0,2.73,2.73,0.333333,0.200000,2.733333,0.0,2.733333,2.733333


## Career Average

In [34]:
# Week 1 stats are player's career average through 2014 season
# Populate current dataframe with current year averages of players

def career_average (grouped_df1, grouped_df2=None):
    new_grouped_df1 = pd.DataFrame(index= grouped_df1.index)
    for col in grouped_df1: # Iterate through every attribute in the team_stats dataframe
        points=[]  # Create an empty list to hold the season average of the current attribute

        try:
            for player in grouped_df1.index.levels[0]:  # Iterate through every player in the team_stats df
                if player in list(grouped_df2.index.levels[0].unique()):
                    total =[grouped_df2.loc[player][col][-1]] # If player not a rookie, use final average from 2015
                else:
                    total =[]  # create an empty list to hold the weekly attributes value
                
                for week in grouped_df1.loc[player].index: # Iterate throough every week for the current player
                    total.append(grouped_df1.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
                    points.append(round(np.mean(total),2))  # Add the average of the total list to the season average list
                    
            new_grouped_df1['{}_car'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe


                                
        except:
            for player in grouped_df1.index.levels[0]:  # Iterate through every player in the team_stats df
                total = []

                for week in grouped_df1.loc[player].index: # Iterate throough every week for the current player
                    total.append(grouped_df1.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
                    points.append(round(np.mean(total),2))  # Add the average of the total list to the season average list

            new_grouped_df1['{}_car'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe
    return new_grouped_df1

In [35]:
df_career_2012 = career_average(te_group['player_2012'])
df_career_2013 = career_average(te_group['player_2013'], te_group['player_2012'])
df_career_2014 = career_average(te_group['player_2014'], te_group['player_2013'])
df_career_2015 = career_average(te_group['player_2015'], te_group['player_2014'])
df_career_2016 = career_average(te_group['player_2016'], te_group['player_2015'])
df_career_2017 = career_average(te_group['player_2017'], te_group['player_2016'])
df_career_2018 = career_average(te_group['player_2018'], te_group['player_2017'])

In [36]:
player_career_grp = pd.concat([df_career_2012, df_career_2013])
player_career_grp = pd.concat([player_career_grp, df_career_2014])
player_career_grp = pd.concat([player_career_grp, df_career_2015])
player_career_grp = pd.concat([player_career_grp, df_career_2016])
player_career_grp = pd.concat([player_career_grp, df_career_2017])
player_career_grp = pd.concat([player_career_grp, df_career_2018])
player_career_grp.head()



tar_car  rec_car  yds_car  td_car  \
Name            games Opponent                                      
Aaron Hernandez 1     TEN          7.00     6.00    59.00    1.00   
                2     ARI          4.00     3.00    29.50    0.50   
                3     SEA          5.67     4.00    29.67    0.67   
                4     NYJ          6.00     4.25    35.75    0.50   
                5     NYJ          5.40     3.80    35.80    0.40   

                                yds/tar_car  yds/rec_car  
Name            games Opponent                            
Aaron Hernandez 1     TEN              8.40         9.80  
                2     ARI              4.20         4.90  
                3     SEA              3.90         4.93  
                4     NYJ              4.85         6.40  
                5     NYJ              6.28         8.72

In [37]:
career_shifted = shift_dict(player_career_grp)
career_shifted.head()

games Opponent  tar_car  rec_car  yds_car  td_car  \
AD Denham 2946    0.0        0      0.0      0.0      0.0     0.0   
          2947    1.0      JAX      0.0      0.0      0.0     0.0   
          2948    2.0      IND      0.0      0.0      0.0     0.0   
          2949    3.0      BAL      0.0      0.0      0.0     0.0   
AJ Derby  5874    0.0        0      0.0      0.0      0.0     0.0   

                yds/tar_car  yds/rec_car  
AD Denham 2946          0.0          0.0  
          2947          0.0          0.0  
          2948          0.0          0.0  
          2949          0.0          0.0  
AJ Derby  5874          0.0          0.0

In [38]:
career_shifted.reset_index(inplace=True)
career_shifted.head()


,level_0,level_1,games,Opponent,tar_car,rec_car,yds_car,td_car,yds/tar_car,yds/rec_car
0,AD Denham,2946,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,AD Denham,2947,1.0,JAX,0.0,0.0,0.0,0.0,0.0,0.0
2,AD Denham,2948,2.0,IND,0.0,0.0,0.0,0.0,0.0,0.0
3,AD Denham,2949,3.0,BAL,0.0,0.0,0.0,0.0,0.0,0.0
4,AJ Derby,5874,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
career_shifted[career_shifted['level_0']=='Rob Gronkowski']

,level_0,level_1,games,Opponent,tar_car,rec_car,yds_car,td_car,yds/tar_car,yds/rec_car
7621,Rob Gronkowski,1198,0.0,0,0.00,0.00,0.00,0.00,0.00,0.00
7622,Rob Gronkowski,1199,1.0,TEN,6.00,6.00,60.00,1.00,10.00,10.00
7623,Rob Gronkowski,1200,2.0,ARI,7.50,6.00,67.50,1.00,9.15,11.25
7624,Rob Gronkowski,1201,3.0,BAL,6.00,4.67,52.00,0.67,8.43,11.00
7625,Rob Gronkowski,1202,4.0,BUF,7.25,4.75,65.00,0.75,8.70,13.45
7626,Rob Gronkowski,1203,5.0,DEN,6.80,4.60,59.00,0.60,8.36,12.52
7627,Rob Gronkowski,1204,6.0,SEA,7.17,4.83,59.33,0.50,8.10,12.13
7628,Rob Gronkowski,1205,7.0,NYJ,7.43,5.00,62.00,0.71,8.19,12.26
7629,Rob Gronkowski,1206,8.0,LAR,8.12,5.38,72.50,0.88,8.56,13.00
7630,Rob Gronkowski,1207,9.0,BUF,7.67,5.11,67.89,0.89,8.48,12.70


In [50]:
career_shifted.drop(['level_0','level_1','games','Opponent'], axis =1, inplace=True)
career_shifted.head()

,tar_car,rec_car,yds_car,td_car,yds/tar_car,yds/rec_car
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
temp.head()

,year,Name,games,Opponent,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,tar_ma,rec_ma,yds_ma,td_ma,yds/tar_ma,yds/rec_ma
0,2014,AD Denham,1,JAX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,AD Denham,2,IND,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,AD Denham,3,BAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,AD Denham,4,JAX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,AJ Derby,1,ARI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
career_shifted.columns

Index(['tar_car', 'rec_car', 'yds_car', 'td_car', 'yds/tar_car',
       'yds/rec_car'],
      dtype='object')

In [153]:
career_shifted = career_shifted.sort_values(['level_0','games_car']).drop(['level_0','level_1','Week','Opponent'], axis=1)
career_shifted.head()

,games_car,att_car,comp%_car,yds_car,td_car,def_rk_car
0,0.0,0.00,0.0,0.00,0.0,0.00
1,0.0,0.00,0.0,0.00,0.0,0.00
2,1.0,44.00,68.0,303.00,2.0,5.00
3,1.5,38.00,68.5,261.00,1.5,18.00
4,2.0,38.33,68.0,248.33,1.0,12.33


In [ ]:
temp1 = pd.concat([temp, career_shifted], axis=1)

In [53]:
temp1.isnull().sum()

year           0
Name           0
games          0
Opponent       0
tar_avg        0
rec_avg        0
yds_avg        0
td_avg         0
yds/tar_avg    0
yds/rec_avg    0
tar_ma         0
rec_ma         0
yds_ma         0
td_ma          0
yds/tar_ma     0
yds/rec_ma     0
tar_car        0
rec_car        0
yds_car        0
td_car         0
yds/tar_car    0
yds/rec_car    0
dtype: int64

In [54]:
temp1.head()

,year,Name,games,Opponent,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,...,yds_ma,td_ma,yds/tar_ma,yds/rec_ma,tar_car,rec_car,yds_car,td_car,yds/tar_car,yds/rec_car
0,2014,AD Denham,1,JAX,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,AD Denham,2,IND,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,AD Denham,3,BAL,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,AD Denham,4,JAX,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,AJ Derby,1,ARI,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
rb.head()

,games,year,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,points
0,1,2018,Aaron Jones,GB,3,WAS,6,42,0,1,5,15,0.0,1.0,5.7
1,2,2018,Aaron Jones,GB,4,BUF,11,65,1,1,17,18,1.0,1.0,15.2
2,3,2018,Aaron Jones,GB,5,DET,7,40,0,3,19,29,0.0,0.0,7.9
3,4,2018,Aaron Jones,GB,6,SF,8,41,0,1,0,22,0.0,1.0,4.1
4,5,2018,Aaron Jones,GB,8,LAR,12,86,1,2,0,14,0.0,1.0,16.6


In [58]:
temp1['surface_0']=te['surface_0'].values
temp1['h/a_1']=te['h/a_1'].values
temp1['def_rk'] = te['def_rk']
temp1['points'] = te['points']

In [59]:
temp1.tail()

,year,Name,games,Opponent,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,...,tar_car,rec_car,yds_car,td_car,yds/tar_car,yds/rec_car,surface_0,h/a_1,def_rk,points
9233,2014,Zach Sudfeld,26,MIA,0.27,0.09,1.36,0.0,1.36,1.36,...,0.33,0.17,1.83,0.0,1.83,1.83,0.0,1.0,24,2.0
9234,2014,Zach Sudfeld,27,MIN,0.33,0.17,2.92,0.0,2.92,2.92,...,0.38,0.23,3.23,0.0,3.23,3.23,0.0,1.0,29,0.0
9235,2014,Zach Sudfeld,28,TEN,0.31,0.15,2.69,0.0,2.69,2.69,...,0.36,0.21,3.00,0.0,3.00,3.00,1.0,0.0,22,0.0
9236,2014,Zach Sudfeld,29,NE,0.29,0.14,2.50,0.0,2.50,2.50,...,0.33,0.20,2.80,0.0,2.80,2.80,0.0,1.0,27,0.6
9237,2014,Zach Sudfeld,30,MIA,0.33,0.20,2.73,0.0,2.73,2.73,...,0.38,0.25,3.00,0.0,3.00,3.00,1.0,0.0,24,4.4


In [60]:
te_stats = temp1
te_stats.shape

(9238, 26)

In [65]:
#A function that will covert a time series database into a supervised learning database

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [160]:
# A function to insert the non reframed columns back into the stats dataframe 

def insert_non_reframed(df, col_dict):
    non_reframed = ['games','year','Opponent','def_rk','h/a_1','surface_0','points']
    for player in df['Name'].unique():
        for col in non_reframed:
            reframed_dict['{}_reframed'.format(player)][col] = col_dict['{}'.format(player)][col].values
            
    return reframed_dict

In [61]:
te_stats.head()

,year,Name,games,Opponent,tar_avg,rec_avg,yds_avg,td_avg,yds/tar_avg,yds/rec_avg,...,tar_car,rec_car,yds_car,td_car,yds/tar_car,yds/rec_car,surface_0,h/a_1,def_rk,points
0,2014,AD Denham,1,JAX,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,30,0.0
1,2014,AD Denham,2,IND,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0.0
2,2014,AD Denham,3,BAL,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4,0.0
3,2014,AD Denham,4,JAX,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30,0.0
4,2016,AJ Derby,1,ARI,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0


In [62]:
te_stats.to_csv('input/te/te_stats.csv', index=False)

In [63]:
te_stats= pd.read_csv('input/te/te_stats.csv')


In [64]:
te_stats.shape

(9238, 26)

In [65]:
te_stats.isnull().sum()

year           0
Name           0
games          0
Opponent       0
tar_avg        0
rec_avg        0
yds_avg        0
td_avg         0
yds/tar_avg    0
yds/rec_avg    0
tar_ma         0
rec_ma         0
yds_ma         0
td_ma          0
yds/tar_ma     0
yds/rec_ma     0
tar_car        0
rec_car        0
yds_car        0
td_car         0
yds/tar_car    0
yds/rec_car    0
surface_0      0
h/a_1          0
def_rk         0
points         0
dtype: int64